In [1]:
import os
import json
import argparse
import time
import math
import numpy as np
from scipy.special import softmax
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import pickle
from gcn.gcn_model import ResidualGatedGCNModel

import tqdm
from tqdm import trange
import tempfile
from sklearn.utils.class_weight import compute_class_weight
from subprocess import check_call


from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

from multiprocessing import Pool
from multiprocessing import cpu_count



In [19]:
def tsp_instance_reader(tspinstance, buff, num_node = 20, withlabel=False):   
    derivate = tspinstance.split(' ')
    buff[:, 0] = np.array(derivate[0:2*num_node:2], dtype = np.float64)
    buff[:, 1] = np.array(derivate[1:2*num_node:2], dtype = np.float64)
    if withlabel:
        opt_sol = np.array(derivate[2*num_node+1:-1], dtype = np.int32) - 1
    else:
        opt_sol= np.array(derivate[2*num_node+1:-1], dtype = np.int32)
    return buff, opt_sol

In [20]:
def write_instance(instance, instance_name, instance_filename):
    with open(instance_filename, "w") as f:
        f.write("NAME : " + instance_name + "\n")
        f.write("COMMENT : blank\n")
        f.write("TYPE : TSP\n")
        f.write("DIMENSION : " + str(len(instance)) + "\n")
        f.write("EDGE_WEIGHT_TYPE : EUC_2D\n")
        f.write("NODE_COORD_SECTION\n")
        s = 1000000
        for i in range(len(instance)):
            f.write(" " + str(i + 1) + " " + str(instance[i][0] * s)[:10] + " " + str(instance[i][1] * s)[:10] + "\n")
        f.write("EOF\n")

def write_para(dataset_name, instance_name, instance_filename, method, para_filename, max_trials=1000, seed=1234):
    with open(para_filename, "w") as f:
        f.write("PROBLEM_FILE = " + instance_filename + "\n")
        f.write("MAX_TRIALS = " + str(max_trials) + "\n")
        f.write("MOVE_TYPE = 5\nPATCHING_C = 3\nPATCHING_A = 2\nRUNS = 1\n")
        f.write("SEED = " + str(seed) + "\n")
        if method == "GCN_LKH":
            f.write("SUBGRADIENT = NO\n")
            f.write("CANDIDATE_FILE = result/" + dataset_name + "/candidate/" + instance_name + ".txt\n")
            f.write("Pi_FILE = result/" + dataset_name + "/pi/" + instance_name + ".txt\n")
        elif method == "FeatGenerate":
            f.write("GerenatingFeature\n")
            f.write("Feat_FILE = result/" + dataset_name + "/feat/" + instance_name + ".txt\n")
        else:
            assert method == "LKH"

In [21]:
def write_candidate_pi(dataset_name, instance_name, candidate, pi):
    n_node = candidate.shape[0]
    with open("result/" + dataset_name + "/candidate/" + instance_name + ".txt", "w") as f:
        f.write(str(n_node) + "\n")
        for j in range(n_node):
            line = str(j + 1) + " 0 5"
            for _ in range(5):
                line += " " + str(int(candidate[j, _]) + 1) + " " + str(_ * 100)
            f.write(line + "\n")
        f.write("-1\nEOF\n")
    with open("result/" + dataset_name + "/pi/" + instance_name + ".txt", "w") as f:
        f.write(str(n_node) + "\n")
        for j in range(n_node):
            line = str(j + 1) + " " + str(int(pi[j]))
            f.write(line + "\n")
        f.write("-1\nEOF\n")

In [22]:
def read_feat(feature_filename):
    edge_index = []
    edge_feature = []
    inverse_edge_index = []
    with open(feature_filename, "r") as f:
        lines = f.readlines()
        for line in lines[:-1]:
            line = line.strip().split()
            for i in range(20):
                edge_index.append(int(line[i * 3]))
                edge_feature.append(int(line[i * 3 + 1]) / 1000000)
                inverse_edge_index.append(int(line[i * 3 + 2]))
    edge_index = np.array(edge_index).reshape(1, -1, 20)
    edge_feature = np.array(edge_feature).reshape(1, -1, 20)
    inverse_edge_index = np.array(inverse_edge_index).reshape(1, -1, 20)
    runtime = float(lines[-1].strip())
    return edge_index, edge_feature, inverse_edge_index, runtime

def read_results(log_filename, max_trials):
    objs = []
    runtimes = []
    with open(log_filename, "r") as f:
        lines = f.readlines()
        for line in lines: # read the obj and runtime for each trial
            if line[:6] == "-Trial":
                line = line.strip().split(" ")
                assert len(objs) + 1 == int(line[-3])
                objs.append(int(line[-2]))
                runtimes.append(float(line[-1]))
        final_obj = int(lines[-6].split(",")[0].split(" ")[-1])
        if len(objs) == 0: # solved by subgradient optimization
            ascent_runtime = float(lines[69].split(" ")[-2])
            return [final_obj] * max_trials, [ascent_runtime]* max_trials
        else:
            assert objs[-1] == final_obj
            return objs, runtimes

def generate_feat(dataset_name, instance, instance_name):
    para_filename = "result/" + dataset_name + "/featgen_para/" + instance_name + ".para"
    instance_filename = "result/" + dataset_name + "/tsp/" + instance_name + ".tsp"
    feature_filename = "result/" + dataset_name + "/feat/" + instance_name + ".txt"
    write_instance(instance, instance_name, instance_filename)
    write_para(dataset_name, instance_name, instance_filename, "FeatGenerate", para_filename)
    with tempfile.TemporaryFile() as f:
        check_call(["./LKH", para_filename], stdout=f)
    return read_feat(feature_filename)

def method_wrapper(args):
    if args[0] == "LKH":
        return solve_LKH(*args[1:])
    elif args[0] == "GCN_LKH":
        return solve_GCN_LKH(*args[1:])
    elif args[0] == "FeatGen":
        return generate_feat(*args[1:])

def solve_LKH(dataset_name, instance, instance_name, rerun=False, max_trials=1000):
    para_filename = "result/" + dataset_name + "/LKH_para/" + instance_name + ".para"
    log_filename = "result/" + dataset_name + "/LKH_log/" + instance_name + ".log"
    instance_filename = "result/" + dataset_name + "/tsp/" + instance_name + ".tsp"
    if rerun or not os.path.isfile(log_filename):
        write_instance(instance, instance_name, instance_filename)
        write_para(dataset_name, instance_name, instance_filename, "LKH", para_filename, max_trials=max_trials)
        with open(log_filename, "w") as f:
            check_call(["./LKH", para_filename], stdout=f)
    return read_results(log_filename, max_trials)



def infer_GCN(net, dataset_node_feat, dataset_edge_index, dataset_edge_feature, dataset_inverse_edge_index, batch_size=100):
    candidate = []
    pi = []
    for i in trange(dataset_edge_index.shape[0] // batch_size):
        node_feat = dataset_node_feat[i * batch_size:(i + 1) * batch_size]
        edge_index = dataset_edge_index[i * batch_size:(i + 1) * batch_size]
        edge_feature = dataset_edge_feature[i * batch_size:(i + 1) * batch_size]
        inverse_edge_index = dataset_inverse_edge_index[i * batch_size:(i + 1) * batch_size]
        node_feat = Variable(torch.FloatTensor(node_feat).type(torch.cuda.FloatTensor), requires_grad=False)
        edge_feature = Variable(torch.FloatTensor(edge_feature).type(torch.cuda.FloatTensor), requires_grad=False).view(batch_size, -1, 1)
        edge_index = Variable(torch.FloatTensor(edge_index).type(torch.cuda.FloatTensor), requires_grad=False).view(batch_size, -1)
        inverse_edge_index = Variable(torch.FloatTensor(inverse_edge_index).type(torch.cuda.FloatTensor), requires_grad=False).view(batch_size, -1)
        y_edges, _, y_nodes = net.forward(node_feat, edge_feature, edge_index, inverse_edge_index, None, None, 20)
        pi.append(y_nodes.cpu().numpy())
        y_edges = y_edges.detach().cpu().numpy()
        y_edges = y_edges[:, :, 1].reshape(batch_size, dataset_node_feat.shape[1], 20)
        y_edges = np.argsort(-y_edges, -1)
        edge_index = edge_index.cpu().numpy().reshape(-1, y_edges.shape[1], 20)
        candidate_index = edge_index[np.arange(batch_size).reshape(-1, 1, 1), np.arange(y_edges.shape[1]).reshape(1, -1, 1), y_edges]
        candidate.append(candidate_index[:, :, :5])
    candidate = np.concatenate(candidate, 0)
    pi = np.concatenate(pi, 0)
    candidate_Pi = np.concatenate([candidate.reshape(dataset_edge_index.shape[0], -1), 1000000 * pi.reshape(dataset_edge_index.shape[0], -1)], -1)
    return candidate_Pi

def solve_GCN_LKH(dataset_name, instance, instance_name, candidate, pi, rerun=False, max_trials=1000):
    para_filename = "result/" + dataset_name + "/GCN_LKH_para/" + instance_name + ".para"
    log_filename = "result/" + dataset_name + "/GCN_LKH_log/" + instance_name + ".log" 
    instance_filename = "result/" + dataset_name + "/tsp/" + instance_name + ".tsp"
    if rerun or not os.path.isfile(log_filename): 
        # write_instance(instance, instance_name, instance_filename)
        write_para(dataset_name, instance_name, instance_filename, "GCN_LKH", para_filename, max_trials=max_trials)
        write_candidate_pi(dataset_name, instance_name, candidate, pi)
        with open(log_filename, "w") as f:
            check_call(["./LKH", para_filename], stdout=f)
    return read_results(log_filename, max_trials)



def eval_dataset(dataset_filename, method, args, rerun=True, max_trials=1000):
    dataset_name = dataset_filename[:-4].split("/")[-1]
    os.makedirs("result/" + dataset_name + "/" + method + "_para", exist_ok=True) 
    os.makedirs("result/" + dataset_name + "/" + method + "_log", exist_ok=True) 
    os.makedirs("result/" + dataset_name + "/tsp", exist_ok=True)
    num_nodes = 2000
    f = open('./data/test/tsp{}_test.txt'.format(num_nodes), 'r')
    data_set = f.readlines()
    dataset=[]
    f.close()
    buff = np.zeros(shape=(num_nodes, 2), dtype = np.float64)
    for i in range(len(data_set)):
        coor, opt = tsp_instance_reader(tspinstance=data_set[i],buff =buff, num_node=num_nodes)
        dataset.append(coor)
    if method == "GCN_LKH":
        os.makedirs("result/" + dataset_name + "/featgen_para", exist_ok=True) 
        os.makedirs("result/" + dataset_name + "/feat", exist_ok=True)
        with Pool(os.cpu_count()) as pool:
            feats = list(tqdm.tqdm(pool.imap(method_wrapper, [("FeatGen", dataset_name, dataset[i], str(i)) for i in range(len(dataset))]), total=len(dataset)))
        feats = list(zip(*feats))
        edge_index, edge_feature, inverse_edge_index, feature_runtime = feats
        feature_runtime = np.sum(feature_runtime)
        edge_index = np.concatenate(edge_index)
        edge_feature = np.concatenate(edge_feature)
        inverse_edge_index = np.concatenate(inverse_edge_index)
        net = ResidualGatedGCNModel()
        net.cuda()
        
        if torch.cuda.is_available():
            checkpoint = torch.load("logs/tsp50/best_val_checkpoint.tar")
        else:
            checkpoint = torch.load("logs/tsp50/best_val_checkpoint.tar", map_location='cpu') 
        net.load_state_dict(checkpoint['model_state_dict'])
        gcn_start_time = time.time()
        with torch.no_grad():
            candidate_Pi = infer_GCN(net, np.array(dataset), edge_index, edge_feature, inverse_edge_index, batch_size=args.batch_size)
        gcn_runtime = time.time() - gcn_start_time
        # with open("candidate_Pi/" + dataset_name + ".pkl", "rb") as f:
        #     candidate_Pi = pickle.load(f)
        n_node = len(dataset[0])
        candidate = candidate_Pi[:, :n_node * 5].reshape(-1, n_node, 5)
        pi = candidate_Pi[:, n_node * 5:].reshape(-1, n_node)
        os.makedirs("result/" + dataset_name + "/candidate", exist_ok=True)
        os.makedirs("result/" + dataset_name + "/pi", exist_ok=True)
        with Pool(os.cpu_count()) as pool:
            results = list(tqdm.tqdm(pool.imap(method_wrapper, [("GCN_LKH", dataset_name, dataset[i], str(i), candidate[i], pi[i], rerun, max_trials) for i in range(len(dataset))]), total=len(dataset)))
    else:
        assert method == "LKH"
        feature_runtime = 0
        gcn_runtime = 0
        with Pool(os.cpu_count()) as pool:
            results = list(tqdm.tqdm(pool.imap(method_wrapper, [("LKH", dataset_name, dataset[i], str(i), rerun, max_trials) for i in range(len(dataset))]), total=len(dataset)))
    results = np.array(results)
    dataset_objs = results[:, 0, :].mean(0)
    dataset_runtimes = results[:, 1, :].sum(0)
    return dataset_objs, dataset_runtimes, feature_runtime, gcn_runtime   

In [ ]:
lkh_objs, lkh_runtimes, _, _ = eval_dataset(args.dataset, "LKH", args=args, rerun=True, max_trials=args.lkh_trials)
gcn_lkh_objs, gcn_lkh_runtimes, feat_runtime, gcn_runtime = eval_dataset(args.dataset, "GCNModel+LKH", args=args, rerun=True, max_trials=args.gcn_lkh_trials)
print ("generating features runtime: %.1fs GCN inferring runtime: %.1fs" % (feat_runtime, gcn_runtime))
trials = 1
while trials <= lkh_objs.shape[0]:
    print ("------experiments of trials: %d ------" % (trials))
    print ("LKH      %d %ds" % (lkh_objs[trials - 1], lkh_runtimes[trials - 1]))
    if trials > gcn_lkh_objs.shape[0]:
        print ("GCNModel+LKH %d %ds (%d trials)" % (gcn_lkh_objs[-1], gcn_lkh_runtimes[-1] + feat_runtime + gcn_runtime, gcn_lkh_objs.shape[0]))
    else:
        print ("GCNModel+LKH %d %ds" % (gcn_lkh_objs[trials - 1], gcn_lkh_runtimes[trials - 1] + feat_runtime + gcn_runtime))
    trials *= 10

print ("------comparison with same time limit------")
trials = 1
while trials <= lkh_objs.shape[0]:
    print ("------experiments of trials: %d ------" % (trials))
    print ("LKH      %d %ds" % (lkh_objs[trials - 1], lkh_runtimes[trials - 1]))
    gcn_lkh_trials = 1
    while gcn_lkh_trials < gcn_lkh_runtimes.shape[0] and gcn_lkh_runtimes[gcn_lkh_trials - 1] + feat_runtime + gcn_runtime < lkh_runtimes[trials - 1]:
        gcn_lkh_trials += 1
    print ("GCNModel+LKH %d %ds (%d trials)" % (gcn_lkh_objs[gcn_lkh_trials - 1], gcn_lkh_runtimes[gcn_lkh_trials - 1] + feat_runtime + gcn_runtime, gcn_lkh_trials))
    trials *= 10

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

In [23]:
args = {'dataset':'data/test/tsp2000_test.txt',
        'model_path':'logs/tsp50/best_val_checkpoint.tar',
        'n_samples':128,
        'batch_size':16,
        'lkh_trials':1000,
        'gcn_lkh_trials':1000
        }
args=argparse.Namespace(**args)

In [32]:
dataset_name = args.dataset[:-4].split("/")[-1]
method="LKH"
os.makedirs("result/" + dataset_name + "/" + method + "_para", exist_ok=True) 
os.makedirs("result/" + dataset_name + "/" + method + "_log", exist_ok=True) 
os.makedirs("result/" + dataset_name + "/tsp", exist_ok=True)
num_nodes = 2000
f = open('./data/test/tsp{}_test.txt'.format(num_nodes), 'r')
data_set = f.readlines()
dataset=[]
f.close()
buff = np.zeros(shape=(num_nodes, 2), dtype = np.float64)
for i in range(len(data_set)):
    coor, opt = tsp_instance_reader(tspinstance=data_set[0],buff =buff, num_node=num_nodes)
    dataset.append(coor)

In [34]:
instance=dataset[0]
instance_name=0
rerun=True
for i in range(len(dataset)):
    instance=dataset[i]
    instance_name=str(i)
    para_filename = "result/" + dataset_name + "/LKH_para/" + instance_name + ".para"
    log_filename = "result/" + dataset_name + "/LKH_log/" + instance_name + ".log"
    instance_filename = "result/" + dataset_name + "/tsp/" + instance_name + ".tsp"
    if rerun or not os.path.isfile(log_filename):
        write_instance(instance, instance_name, instance_filename)
        write_para(dataset_name, instance_name, instance_filename, "LKH", para_filename, max_trials=1000)